In [ ]:
from mistral_common.protocol.instruct.messages import AssistantMessage, ToolMessage, UserMessage
from mistral_common.protocol.instruct.tool_calls import Function, FunctionCall, Tool, ToolCall
from mistral_common.tokens.instruct.normalize import ChatCompletionRequest
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

mistral_tokenizer = MistralTokenizer.v3()

completion_request = ChatCompletionRequest(
    tools=[
        Tool(
            function=Function(
                name="get_current_weather",
                description="Get the current weather",
                parameters={
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "format": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The temperature unit to use. Infer this from the users location.",
                        },
                    },
                    "required": ["location", "format"],
                },
            )
        )
    ],
    messages=[
        UserMessage(content="What's the weather like today in Paris?"),
        AssistantMessage(
            tool_calls=[
                ToolCall(
                    id="1bdc45f90",
                    function=FunctionCall(name="get_weather", arguments='{"location": "Tokyo, JP", "unit": "celsius"}'),
                ),
            ]
        ),
        ToolMessage(
            content="Weather in Tokyo, JP: Partly cloudy, 21 degrees celsius.",
            tool_call_id="1bdc45f90",
        ),
        AssistantMessage(content="It's partly cloudy and 21 degrees in Tokyo."),
        UserMessage(content="What's the weather like today in Paris?"),
        AssistantMessage(content="It's partly cloudy and 21 degrees in Tokyo."),
        UserMessage(content="What's the weather like today in Paris?"),
    ],
)

tokenized = mistral_tokenizer.encode_chat_completion(completion_request)

print(tokenized.text)

In [ ]:
from transformers import AutoTokenizer

hf_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

In [ ]:
test = """<s>[INST] What's the weather in Tokyo?[/INST] [TOOL_CALLS] [{"name": "get_weather", "arguments": {"location": "Tokyo, JP", "unit": "celsius"}, "id": "dccf5329c"}]</s>[TOOL_RESULTS] {"call_id": "dccf5329c", "content": "Weather in Tokyo, JP: Partly cloudy, 21 degrees celsius."}[/TOOL_RESULTS] It's partly cloudy and 21 degrees in Tokyo.</s>"""
hf_tokenizer(test)